<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/mlops-world/dynamic-quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Dynamic Quantization with Hugging Face Optimum

In this session, you will learn how to apply _dynamic quantization_ to a 🤗 Transformers model. You will quantize a [DistilBERT model](https://huggingface.co/optimum/distilbert-base-uncased-finetuned-banking77) that's been fine-tuned on the [Banking77 dataset](https://huggingface.co/datasets/banking77) for intent classification. 

Along the way, you'll learn how to use two open-source libraries: 

* [🤗 Optimum](https://github.com/huggingface/optimum): an extension of 🤗 Transformers, which provides a set of performance optimization tools enabling maximum efficiency to train and run models on targeted hardware.
* [🤗 Evaluate](https://github.com/huggingface/evaluate): a library that makes evaluating and comparing models and reporting their performance easier and more standardized.


By the end of this session, you see how quantization with 🤗 Optimum can significantly decrease model latency while keeping almost 100% of the full-precision model.


> This tutorial was created and run on a c6i.xlarge AWS EC2 Instance.

## Learning objectives

By the end of this session, you will know how to:

* Setup a development environment
* Convert a 🤗 Transformers model to ONNX for inference
* Apply dynamic quantization using `ORTQuantizer` from 🤗 Optimum
* Test inference with the quantized model
* Evaluate the model performance with 🤗 Evaluate
* Compare the latency of the quantized model against the original one
* Push the quantized model to the Hub
* Load and run inference with a quantized model from the Hub


Let's get started! 🚀

## 1. Setup development environment

Our first step is to install 🤗 Optimum, along with 🤗 Evaluate and some other libraries. Running the following cell will install all the required packages for us including 🤗 Transformer, PyTorch, and ONNX Runtime utilities:

In [ ]:
%pip install "optimum[onnxruntime]==1.2.2" "evaluate[evaluator]" sklearn mkl-include mkl

> If you want to run inference on a GPU, you can install 🤗 Optimum with `pip install optimum[onnxruntime-gpu]`.

While we're at it, let's turn off some of the warnings from the 🤗 Datasets library and the tokenizer:

In [1]:
import datasets

datasets.logging.set_verbosity_error()

%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


## 2. Convert a 🤗 Transformers model to ONNX for inference

Before we can optimize and quantize our model, we first need to export it to the ONNX format. To do this we will use the `ORTModelForSequenceClassification` class and call the `from_pretrained()` method. This method will download the PyTorch weights from the Hub and export them via the `from_transformers` argument. The model we are using is `optimum/distilbert-base-uncased-finetuned-banking77`, which is a fine-tuned DistilBERT model on the Banking77 dataset achieving an accuracy score of 92.5% and as the feature (task) text-classification:

In [2]:
from pathlib import Path

from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

model_id = "optimum/distilbert-base-uncased-finetuned-banking77"
dataset_id = "banking77"
onnx_path = Path("onnx")

# load vanilla transformers and convert to onnx
model = ORTModelForSequenceClassification.from_pretrained(
    model_id, from_transformers=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Downloading:   0%|          | 0.00/5.81k [00:00<?, ?B/s]

One neat thing about 🤗 Optimum, is that allows you to run ONNX models with the `pipeline()` function from  🤗 Transformers. This means that you get all the pre- and post-processing features for free, without needing to re-implement them for each model! Here's how you can run inference with our vanilla ONNX model:

In [3]:
from transformers import pipeline

vanilla_clf = pipeline("text-classification", model=model, tokenizer=tokenizer)
vanilla_clf("Could you assist me in finding my lost card?")

[{'label': 'lost_or_stolen_card', 'score': 0.9664045572280884}]

This looks good, so let's save the model and tokenizer to disk for later usage:

In [4]:
# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/vocab.txt',
 'onnx/added_tokens.json',
 'onnx/tokenizer.json')

If we inspect the `onnx` directory where we've saved the model and tokenizer:

In [5]:
!ls {onnx_path}

config.json  special_tokens_map.json  tokenizer_config.json
model.onnx   tokenizer.json	      vocab.txt


we can see that there's a `model.onnx` file that corresponds to our exported model. Let's now go ahead and optimize this!

## 3. Apply dynamic quantization using `ORTQuantizer` from 🤗 Optimum

To apply quantization in 🤗 Optimum, we do this by:

* Creating an optimizer based on our ONNX model
* Defining the type of optimizations via a configuration class
* Exporting the optimized model as a new ONNX file

The following code snippet does these steps for us:

In [6]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# create ORTQuantizer and define quantization configuration
dynamic_quantizer = ORTQuantizer.from_pretrained(model_id, feature=model.pipeline_task)
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model
model_quantized_path = dynamic_quantizer.export(
    onnx_model_path=onnx_path / "model.onnx",
    onnx_quantized_model_output_path=onnx_path / "model-quantized.onnx",
    quantization_config=dqconfig,
)

Here we can see that we've specifed in the configuration the type of execution engine to use with the Intel AVX512-VNNI CPU. If we now take a look at our `onnx` directory:

In [7]:
!ls {onnx_path}

config.json	      model.onnx	       tokenizer.json	      vocab.txt
model-quantized.onnx  special_tokens_map.json  tokenizer_config.json


we can see we have a new ONNX file called `model-quantized-dynamic.onnx`. Let's do a quick size comparison of the two models:

In [8]:
import os

# get model file size
size = os.path.getsize(onnx_path / "model.onnx") / (1024 * 1024)
quantized_model = os.path.getsize(onnx_path / "model-quantized.onnx") / (1024 * 1024)

print(f"Model file size: {size:.2f} MB")
print(f"Quantized Model file size: {quantized_model:.2f} MB")

Model file size: 255.68 MB
Quantized Model file size: 134.23 MB


Nice, dynamic quantization has reduced the model size by around a factor of 2! This should allow us to speed up the inference time by a similar factor, so let's now see how we can test the latency of our models.

## 4. Test inference with the quantized model

As we saw earlier, Optimum has built-in support for transformers pipelines. This allows us to leverage the same API that we know from using PyTorch and TensorFlow models. Therefore we can load our quantized model with `ORTModelForSequenceClassification` class and the transformers `pipeline()` function:

In [9]:
model = ORTModelForSequenceClassification.from_pretrained(
    onnx_path, file_name="model-quantized.onnx"
)
tokenizer = AutoTokenizer.from_pretrained(onnx_path)

quantized_clf = pipeline("text-classification", model=model, tokenizer=tokenizer)
quantized_clf("Could you assist me in finding my lost card?")

[{'label': 'lost_or_stolen_card', 'score': 0.9570642709732056}]

## 5. Evaluate the model performance with 🤗 Evaluate

It is always a good idea to evaluate the performance of your quantized model on a dedicated test set to ensure the optimizations haven't impacted the model too strongly. To evaluate our model, we'll use the handy `evaluator()` function from 🤗 Evaluate. This function is similar to the `pipeline()` function from 🤗 Transformers, in the sense that it handles the evaluation loop for you automatically!

Here's how you can load an evaluator for text classification and feed in the quantized pipeline:

In [10]:
from datasets import load_dataset
from evaluate import evaluator

eval_pipe = evaluator("text-classification")
eval_dataset = load_dataset("banking77", split="test")
label_mapping = model.config.label2id

results = eval_pipe.compute(
    model_or_pipeline=quantized_clf,
    data=eval_dataset,
    metric="accuracy",
    input_column="text",
    label_column="label",
    label_mapping=label_mapping,
    strategy="simple",
)
print(results)

{'accuracy': 0.9243506493506494}


Not bad! The resulting accuracy isn't too far from the original model - let's see how much exactly:

In [11]:
print(f"Vanilla model: 92.5%")
print(f"Quantized model: {results['accuracy']*100:.2f}%")
print(
    f"The quantized model achieves {round(results['accuracy']/0.925,4)*100:.2f}% accuracy of the fp32 model"
)

Vanilla model: 92.5%
Quantized model: 92.44%
The quantized model achieves 99.93% accuracy of the fp32 model


## 6. Compare the latency of the quantized model against the original one

Okay, now let's test the performance (latency) of our quantized model. We are going to use a payload with a sequence length of 128 for the benchmark. To keep it simple, we are going to use a Python loop and calculate the avgerage and p95 latencies for our vanilla model and for the quantized model:

In [11]:
from time import perf_counter

import numpy as np

payload = (
    "Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer this email in the next 7 days. Best regards and have a nice weekend "
    * 2
)
print(f'Payload sequence length: {len(tokenizer(payload)["input_ids"])}')


def measure_latency(pipe):
    latencies = []
    # warm up
    for _ in range(10):
        _ = pipe(payload)
    # Timed run
    for _ in range(300):
        start_time = perf_counter()
        _ = pipe(payload)
        latency = perf_counter() - start_time
        latencies.append(latency)
    # Compute run statistics
    time_avg_ms = 1000 * np.mean(latencies)
    time_std_ms = 1000 * np.std(latencies)
    time_p95_ms = 1000 * np.percentile(latencies, 95)
    return (
        f"P95 latency (ms) - {time_p95_ms}; Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f};",
        time_p95_ms,
    )


vanilla_model = measure_latency(vanilla_clf)
quantized_model = measure_latency(quantized_clf)

print(f"Vanilla model: {vanilla_model[0]}")
print(f"Quantized model: {quantized_model[0]}")
print(
    f"Improvement through quantization: {round(vanilla_model[1]/quantized_model[1],2)}x"
)

Payload sequence length: 128
Vanilla model: P95 latency (ms) - 63.24214749874954; Average latency (ms) - 62.87 +\- 0.85;
Quantized model: P95 latency (ms) - 37.06475604940351; Average latency (ms) - 37.93 +\- 9.72;
Improvement through quantization: 1.71x


Nice, our model is model is almost two times faster, despite losing little to no accuracy!

![](images/dynamic-quantization.png)

## 6. Push the quantized model to the Hub

The Optimum model classes like `ORTModelForSequenceClassification` are integrated with the Hugging Face Model Hub, which means you can not only load model from the Hub, but also push your models to the Hub with the `push_to_hub()` method. That way we can now save our qunatized model on the Hub to be for example used inside our inference API.

We have to make sure that we are also saving the tokenizer as well as the `config.json` to have a good inference experience.

If you haven't logged into the Hub yet you can use the `notebook_login` function to do so:

In [17]:
from huggingface_hub import notebook_login

notebook_login()

It's then a simple mater of saving our files to a local directory and running the `push_to_hub()` method:

In [13]:
tmp_store_directory = "onnx_hub_repo"
repository_id = "quantized-distilbert-banking77"
model_file_name = "model-quantized.onnx"

model.latest_model_name = model_file_name  # workaround for PR #214
model.save_pretrained(tmp_store_directory)
dynamic_quantizer.tokenizer.save_pretrained(tmp_store_directory)

model.push_to_hub(tmp_store_directory, repository_id=repository_id, use_auth_token=True)

/home/ubuntu/miniconda3/envs/hf/lib/python3.9/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.8. Pass `repo_id` instead.
  warnings.warn(


## 7. Load and run inference from the Hub

Now that our model is on the Hub, we can use it from anywhere! Here's a demo to show how we can load the model and tokenizer, before passing them to the `pipeline()` function:

In [14]:
model = ORTModelForSequenceClassification.from_pretrained(
    "lewtun/quantized-distilbert-banking77"
)
tokenizer = AutoTokenizer.from_pretrained("lewtun/quantized-distilbert-banking77")

remote_clf = pipeline("text-classification", model=model, tokenizer=tokenizer)
remote_clf("Could you assist me in finding my lost card?")

Downloading:   0%|          | 0.00/5.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/141M [00:00<?, ?B/s]

[{'label': 'lost_or_stolen_card', 'score': 0.9570642709732056}]